In [1]:
%matplotlib inline
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
from matplotlib.colors import SymLogNorm
import scienceplots

import astropy.units as u
from astropy import constants
from astropy.table import Table
from astropy.coordinates import SkyCoord,FK5,ICRS
from astropy.io import fits,ascii
from astropy.wcs import WCS, utils as wcsutils
from astropy.stats import sigma_clipped_stats
import astropy.visualization as vis
from astropy.modeling import models, fitting
from astropy.nddata import StdDevUncertainty, Cutout2D
from astropy.convolution import Gaussian2DKernel,Gaussian1DKernel
from astropy.cosmology import Planck18
from scipy.signal import find_peaks
#from scipy.constants import c



from photutils.aperture import EllipticalAperture, SkyEllipticalAperture

import os, sys, glob, pdb, scipy

from importlib import reload
import pyregion as pyreg
from spectral_cube import SpectralCube
from reproject import reproject_interp
import regions
from regions.shapes.circle import CircleSkyRegion,CirclePixelRegion
from regions import CircleAnnulusSkyRegion, CircleAnnulusPixelRegion, RectangleSkyRegion
from regions import PixCoord
sys.path.append('/disk/bifrost/yuanze/KBSS/MUSEQSO/scripts')
import run_cubetools_MUSE as ctools
import makeMask_MUSE as mmask
reload(ctools)
reload(mmask)
sys.path.append('/disk/bifrost/yuanze/software/GalfitS/src')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import images as IM
import sed_interp
reload(IM)

brightu = r"Brightness $\rm (10^{-4}~erg~s^{-1}~arcsec^{-2}~cm^{-2}~\AA^{-1})$"
def imshow_astro(img, wcsinfo = None, figsize = (10,10), colorbar =True,
               cblabel="", cbfrac = 0.035, norm = None,
               stretch = vis.LinearStretch(), cmap = "hot",
               vrange = (None, None)):
    
    _, med, std = sigma_clipped_stats(img.data)
    
    fig =  plt.figure(figsize = figsize)
    if wcsinfo:
        ax = plt.subplot(projection = wcsinfo)
    else:
        ax = plt.subplot()
    
    vmin, vmax = vrange
    
    if not vmin:
        vmin = med
    if not vmax:
        vmax = med + 10*std
    if not norm:
        norm = vis.ImageNormalize(vmin = vmin, vmax = vmax, stretch = stretch)
    im = ax.imshow(img, norm = norm, cmap = cmap)
    if colorbar:
        cb = plt.colorbar(im, label = cblabel, fraction =cbfrac)
    return fig, ax, im
KBSSpath="/disk/bifrost/yuanze/KBSS"
qsos_bright = ascii.read(KBSSpath+"/KCWI/qsos_bright_updated.list",format="ipac")

#MUSEQSO= ascii.read(KBSSpath+"/KCWI/MUSEQSO_machine_readable_updated2.list",format="ipac")
#museqso_path = os.path.join(KBSSpath, "MUSEQSO")
#votable_file = os.path.join("fixed_file.xml")

In [ ]:
wave=np.arange(3500,6500)
zlow=2.65
zhigh=3.15
lowz_igm=sed_interp.igm_transmission(wave,zlow)
highz_igm=sed_interp.igm_transmission(wave,zhigh)
plt.plot(wave/(zlow+1),lowz_igm,label="lowz")
plt.plot(wave/(zhigh+1),highz_igm,label="highz")
lowzlyaind=np.argmin(np.abs(wave/(zlow+1)-1215.67))
highzlyaind=np.argmin(np.abs(wave/(zhigh+1)-1215.67))
print(lowz_igm[lowzlyaind])
print(highz_igm[highzlyaind])
print("correction factor:",lowz_igm[lowzlyaind]/highz_igm[highzlyaind])
plt.axvline(1215.67,lw=1,ls="--")
plt.legend()

0.77595234
0.66500735
correction factor: 1.1668327


In [11]:

reload(mmask)
from datetime import datetime# Making white-light cutout
root_directory = KBSSpath
condition=[qsos_bright['Field']!="Q0142"]
source_table = qsos_bright#[(qsos_bright['Field']!="Q1623") & (qsos_bright['Field']!="Q0142")]

#print("Number of directories found:",len(all_directories))
#gc.set_debug(gc.DEBUG_LEAK)
radius=6
dovar=True
for element in source_table:#[qsos_bright['Field']=="Q0449"]:
    print(element)
    qname = element["Name"]
    field = element["Field"]
    objname="qso"
    subdapath=KBSSpath+"/"+field+"/"+objname.upper()
    #CubRunpath=KBSSpath+"/CubEx_run/"+field
    fn=subdapath+"/{}-{}_icubes_wcs.PSFSub.fits".format(field.lower(),objname,int(radius/0.3))
    hdu0=fits.open(fn)
    hdr = hdu0[0].header
    wc = (np.arange(hdr['NAXIS3']) - hdr['CRPIX3'] + 1) * hdr['CD3_3'] + hdr['CRVAL3']
    #NSubfile = adp_prefix+".PSFSub.fits"
    #zmask = np.loadtxt(subdir+"/zmask.txt").astype(np.int64)
    zmaskmin,zmaskmax = mmask.determine_mask(wc,["continuum"], element['z_sys'])
    xpix = element["x"]
    ypix = element["y"]
    wimage = np.nansum(hdu0[0].data[zmaskmin[0]:zmaskmax[0],:],axis=0)
    wimage[wimage==0] = np.nan
    header = fits.Header()
    header['DATE'] = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    header['CREATOR'] = 'Yuanze Ding'
    header['COMMENT'] = 'White-light cutout from 3D data cube'
    header['YSOURCE'] = ypix
    header['XSOURCE'] = xpix
    header['OBJECT'] = qname
    # Extract WCS information from the original header
    wcs_keys = ['CTYPE1', 'CTYPE2', 'CRVAL1', 'CRVAL2', 'CRPIX1', 'CRPIX2', 'CDELT1', 'CDELT2', 'CUNIT1', 'CUNIT2', 'CD1_1', 'CD1_2', 'CD2_1', 'CD2_2']
    for key in wcs_keys:
        if key in hdr:
            header[key] = hdr[key]

    hdu = fits.PrimaryHDU(wimage,header=header)
    output_fn = subdapath+"/{}-{}_icubes_wcs_{}.PSFSub.white.fits".format(field.lower(),objname,int(radius/0.3))
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(output_fn, overwrite=True)
    if dovar:
        fnvar=subdapath+"/{}-{}_vcubes.fits".format(field.lower(),objname)
        hdu0_var=fits.open(fnvar)
        wvar = np.nansum(hdu0_var[0].data[zmaskmin[0]:zmaskmax[0],:],axis=0)
        wvar[wvar==0] = np.nan
        output_fnvar = subdapath+"/{}-{}_icubes_wcs.whitevar.fits".format(field.lower(),objname)
        hduvar = fits.PrimaryHDU(wvar,header=header)
        hdulist = fits.HDUList([hduvar])
        hdulist.writeto(output_fnvar, overwrite=True)
#hdu2=fits.open(KBSSpath+"/MUSEQSO/Q-0347-383/ADP.2016-06-02T13:21:57.103.fits")

  Name    aka   Field   RA    Decl    x     y    EBV  z_simbad z_sys  zerr contam  u    g    r    i    z      MB    Comments          Mi             z_Lyaneb    
  unit    unit   unit  unit   unit   unit  unit  unit   unit    unit  unit  unit  unit unit unit unit unit   unit     unit                                       
-------- ------ ----- ------ ------ ----- ----- ----- -------- ----- ----- ------ ---- ---- ---- ---- ---- -------- -------- ------------------- ----------------
Q0100+13 PHL957 Q0100 15.797 13.272 52.24 52.55 0.031    2.686 2.721 0.001  False 17.4 16.7 16.5 16.5   -- -26.4472       -- -30.321267391475878 2.72238484467835
Line: continuum_blue, Min index: 1462, Max index: 1508
    Name    aka  Field   RA    Decl    x    y    EBV  z_simbad z_sys  zerr contam  u     g    r     i     z    MB  Comments         Mi              z_Lyaneb     
    unit    unit  unit  unit   unit   unit unit  unit   unit    unit  unit  unit  unit  unit unit  unit  unit unit   unit              

In [12]:
import warnings
from regions import Regions
reload(ctools)
root_directory = KBSSpath
condition=[qsos_bright['Field']!="Q0142"]
#source_table = qsos_bright#[(qsos_bright['Field']!="Q1623") & (qsos_bright['Field']!="Q0142")]
source_table=qsos_bright#[qsos_bright['Field']=="Q0449"]
print("Number of directories found:",len(source_table))
#gc.set_debug(gc.DEBUG_LEAK)
radius=6
cutr=20

for element in source_table:
    qname = element["Name"]
    field = element["Field"]
    print(qname)
    
    objname="qso"
    subdapath=KBSSpath+"/"+field+"/"+objname.upper()
    CubRunpath=KBSSpath+"/CubEx_run/"+field
    ra = element['RA']
    dec = element['Decl']
    sc= SkyCoord(ra,dec,unit=(u.deg,u.deg))
   # qname="LBQS 2139-4434"
    fname=subdapath+"/{}-{}_icubes_wcs_{}.PSFSub.white.fits".format(field.lower(),objname,int(radius/0.3))
    fnamevar=subdapath+"/{}-{}_icubes_wcs.whitevar.fits".format(field.lower(),objname,int(radius/0.3))
    hduvar=fits.open(fnamevar)
    var=hduvar[0].data
    #var[var==0]=np.nan
    mean_sky_var, _, _ = sigma_clipped_stats(var, sigma=2.0, maxiters=None)
    img = IM.image(fname,hdu=0,unit=u.Jy)
    xpix = element["x"]
    ypix = element["y"]
    wcs=WCS(img.data.header)
    
    
    reg= ctools.read_regions(KBSSpath+f"/AbsHosts_nLOS_reg/{field}_AbsHost-nLOS.reg",pix_scale=wcs.pixel_scale_matrix[1,1]*3600)
    artists=[]
    metas=[]
    for r in reg:
        artists.append(r.to_pixel(wcs).as_artist())
        meta=dict(r.meta)
        meta["fontsize"]=5
        metas.append(meta)
    center = [ypix,xpix]
    imcut, cp = img.img_cut(sc.ra.value,sc.dec.value,cutr)
    sky_mean, sky_median, sky_std = sigma_clipped_stats(imcut, sigma=3.0, maxiters=None)
    print("sky level from vcube:",np.sqrt(mean_sky_var),"; from sigma clipping:",sky_std,"; bkg level:",sky_mean)
    img.sources_skycord_cut = [xpix,ypix]
    #img.cut_mask_image = np.zeros_like(img.cut_sigma_image)
    sdmask = ctools.create_circular_mask(img.data.shape, center, 7)
    try:
        with warnings.catch_warnings(record=True) as w:
            warnings.simplefilter("always")
            img.cut_mask_image = img.generate_cutmask(4,nsigma=2,sky_level=np.sqrt(mean_sky_var),sdmask=sdmask,data = img.data.data,deblend=False,addgrow=1,nlevels=32,source_dia=5,contrast=0.001)
    except:
        for warning in w:
            print(f"Warning: {warning.message}")
        continue
    ctools.maskplot(img.data.data,img.cut_mask_image,meta=metas,artist=artists,output=root_directory+"/diagnostic/"+qname+"_maskmap.pdf",center=center)
    outname = subdapath+"/{}-{}_icubes_wcs_{}.PSFSub.mask.fits".format(field.lower(),objname,int(radius/0.3))
    ctools.write_fits_cube(img.cut_mask_image.astype(np.int32),img.data.header,outname)
    #NSubfile = adp_prefix+".PSFSub.fits"
    #zmask = np.loadtxt(subdir+"/zmask.txt",dtype=int)

Number of directories found: 14
Q0100+13


15.797 13.272
51.32391671204359 57.76420000122385
sky level from vcube: 0.045205661804038635 ; from sigma clipping: 0.03575155 ; bkg level: -0.00016164304


HS0105+1619
17.027 16.597
49.44023838716613 50.25180152295992
sky level from vcube: 0.038706077590482814 ; from sigma clipping: 0.03402273 ; bkg level: 0.004930389


Q0207-003
32.461 -0.085
53.584 52.5570000000555
sky level from vcube: 0.03315906496861369 ; from sigma clipping: 0.0348819 ; bkg level: 0.0041410713


Q0449-1645
73.06 -15.329
44.74433547938025 16159.414357424987
sky level from vcube: 0.4032850997164036 ; from sigma clipping: nan ; bkg level: nan


...

HS1700+64
255.252 64.203
50.18466092431497 54.351006240866376


sky level from vcube: 0.0665610812275958 ; from sigma clipping: 0.049031187 ; bkg level: 6.551099e-05


Q2206-199
332.217 -19.733
47.04518740670729 52.81299994657056
sky level from vcube: 0.042672610028643004 ; from sigma clipping: 0.04001933 ; bkg level: 0.0033392338


Q2343+12


356.618 12.816
91.25968518459831 48.62000135054741
sky level from vcube: 0.019830383200545258 ; from sigma clipping: 0.036359973 ; bkg level: 0.007731924


In [9]:
img.cut_mask_image.astype(np.int32)

AttributeError: 'NoneType' object has no attribute 'astype'

In [10]:
fname

'/disk/bifrost/yuanze/KBSS/Q2343/QSO/q2343-qso_icubes_wcs_20.PSFSub.white.fits'

In [81]:
artists[0].center

(50.04648997238098, 49.072736870607734)

In [67]:
reg[0].meta

{'label': 'Q0100  2.721'}

In [13]:
import re
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion, PixCoord
from astropy import units as u

# Read the region file
file_path = KBSSpath+f"/AbsHosts_nLOS_reg/{field}_AbsHost-nLOS.reg"
with open(file_path, 'r') as file:
    lines = file.readlines()

# Regex to extract information
pattern = re.compile(r'fk5;circle\((.*?),(.*?),(.*?)i\) #color=(.*?)  width=2 text={(.*?)}')

regions = []

for line in lines:
    match = pattern.match(line)
    if match:
        ra, dec, radius, color, text = match.groups()
        coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg), frame='fk5')
        radius = float(radius) * u.arcsec  # assuming the radius is in arcseconds
        
        # Create a CircleSkyRegion
        region = CircleSkyRegion(center=coord, radius=radius)
        regions.append(region)

# Now `regions` contains all the parsed CircleSkyRegions
# You can manipulate them or plot them using regions library

# Example: Printing region details
for region in regions:
    print(region)

Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (73.05958333, -16.67116667)>
radius: 3.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (73.059725, -16.67320556)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (73.05902917, -16.67270556)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (73.0596375, -16.67312222)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (73.0579, -16.67265)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (73.05906833, -16.66973056)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (73.05526667, -16.67153889)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equ